In [5]:
# Nurse Assignment problem
# Aziz Gafurov
# 2/24/2025
import numpy as np
np.float_ = np.float64
import pandas as pd
from pandas import DataFrame
from docplex.mp.model import Model



excel_file = "Data_Nurses_Assignment_Problem.xlsx"
nurse_xlsx_file = pd.ExcelFile(excel_file)

In [6]:
df_departments = pd.read_excel(nurse_xlsx_file, sheet_name="Departments")
df_skills = pd.read_excel(nurse_xlsx_file, sheet_name="Skills")
df_shifts = pd.read_excel(nurse_xlsx_file, sheet_name="Shifts")
df_shifts.index.name = "Shift_ID"
df_skill_requirements = pd.read_excel(nurse_xlsx_file, sheet_name="Skill Requirements")
df_nurses = pd.read_excel(nurse_xlsx_file, sheet_name="Nurses", header=0, index_col=0)
df_nurse_skills = pd.read_excel(nurse_xlsx_file, sheet_name="Nurse Skills")
df_nurse_vacations = pd.read_excel(nurse_xlsx_file, sheet_name="Nurse Vacations")
df_nurse_associations = pd.read_excel(nurse_xlsx_file, sheet_name="Nurse Associations")
df_incompatibilities = pd.read_excel(nurse_xlsx_file, sheet_name="Nurse Incompatibilities")

In [7]:
#data exploration


print(f"Number of nurses: {len(df_nurses)}")
print(f"Number of shifts: {len(df_shifts)}")
print(f"Number of vacations: {len(df_nurse_vacations)}")

Number of nurses: 32
Number of shifts: 41
Number of vacations: 59


In [8]:
max_work_time = 40
max_nb_shifts = 5

In [9]:
days = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
day_of_week = dict(zip(days, range(7)))

In [10]:
def day_to_day_of_week(day):
    return day_of_week[day.lower()]

def calculate_absolute_end_time(start, end, day_number):
    return 24*day_number + end + (24 if start >= end else 0)

In [11]:
df_shifts['Day_Number'] = df_shifts['Day'].apply(day_to_day_of_week)
df_shifts['Start_Time_Converted'] = df_shifts['Start_Time'] + df_shifts['Day_Number'] * 24
df_shifts['End_Time_Converted'] = df_shifts.apply(lambda row: calculate_absolute_end_time(row.Start_Time, row.End_Time, row.Day_Number), axis=1)
df_shifts['Duration'] = df_shifts.End_Time_Converted - df_shifts.Start_Time_Converted
df_shifts['Min_Demand_Hours'] = df_shifts.Min_Required * df_shifts.Duration

In [12]:
df_shifts

,Department,Day,Start_Time,End_Time,Min_Required,Max_Required,Day_Number,Start_Time_Converted,End_Time_Converted,Duration,Min_Demand_Hours
Shift_ID,,,,,,,,,,,
0,Emergency,Monday,2,8,3,5,0,2,8,6,18
1,Emergency,Monday,8,12,4,7,0,8,12,4,16
2,Emergency,Monday,12,18,2,5,0,12,18,6,12
3,Emergency,Monday,18,2,3,7,0,18,26,8,24
4,Consultation,Monday,8,12,10,13,0,8,12,4,40
5,Consultation,Monday,12,18,8,12,0,12,18,6,48
6,Cardiac Care,Monday,8,12,10,13,0,8,12,4,40
7,Cardiac Care,Monday,12,18,8,12,0,12,18,6,48
8,Emergency,Tuesday,8,12,4,7,1,32,36,4,16


In [16]:
# building the model
mdl = Model(name="Nurses Assignment Problem")

In [ ]:
all_nurses = df_nurses.index.values
all_shifts = df_shifts.index.values
assigned = mdl.binary_var_matrix(keys1=all_nurses, keys2=all_shifts, name="assign_%s_%s")

In [25]:
df_assigned = DataFrame({"Assigned" : assigned})
df_assigned.index.names = ["All_Nurses", "All_Shifts"]
df_assigned_pivot = df_assigned.unstack(level="All_Shifts")
df_assigned_pivot

Assigned                                        \
All_Shifts                 0                  1                  2    
All_Nurses                                                            
Anne            assign_Anne_0      assign_Anne_1      assign_Anne_2   
Bethanie    assign_Bethanie_0  assign_Bethanie_1  assign_Bethanie_2   
Betsy          assign_Betsy_0     assign_Betsy_1     assign_Betsy_2   
Cathy          assign_Cathy_0     assign_Cathy_1     assign_Cathy_2   
Cecilia      assign_Cecilia_0   assign_Cecilia_1   assign_Cecilia_2   
Chris          assign_Chris_0     assign_Chris_1     assign_Chris_2   
Cindy          assign_Cindy_0     assign_Cindy_1     assign_Cindy_2   
David          assign_David_0     assign_David_1     assign_David_2   
Debbie        assign_Debbie_0    assign_Debbie_1    assign_Debbie_2   
Dee              assign_Dee_0       assign_Dee_1       assign_Dee_2   
Gloria        assign_Gloria_0    assign_Gloria_1    assign_Gloria_2   
Isabelle    assign_Isabelle_0  assign_Isabelle_1  assign_Isabelle_2   
Jane            assign_Jane_0      assign_Jane_1      assign_Jane_2   
Janelle      assign_Janelle_0   assign_Janelle_1   assign_Janelle_2   
Janice        assign_Janice_0    assign_Janice_1    assign_Janice_2   
Jemma          assign_Jemma_0     assign_Jemma_1     assign_Jemma_2   
Joan            assign_Joan_0      assign_Joan_1      assign_Joan_2   
Joyce          assign_Joyce_0     assign_Joyce_1     assign_Joyce_2   
Jude            assign_Jude_0      assign_Jude_1      assign_Jude_2   
Julie          assign_Julie_0     assign_Julie_1     assign_Julie_2   
Juliet        assign_Juliet_0    assign_Juliet_1    assign_Juliet_2   
Kate            assign_Kate_0      assign_Kate_1      assign_Kate_2   
Nancy          assign_Nancy_0     assign_Nancy_1     assign_Nancy_2   
Nathalie    assign_Nathalie_0  assign_Nathalie_1  assign_Nathalie_2   
Nicole        assign_Nicole_0    assign_Nicole_1    assign_Nicole_2   
Patricia    assign_Patricia_0  assign_Patricia_1  assign_Patricia_2   
Patrick      assign_Patrick_0   assign_Patrick_1   assign_Patrick_2   
Roberta      assign_Roberta_0   assign_Roberta_1   assign_Roberta_2   
Suzanne      assign_Suzanne_0   assign_Suzanne_1   assign_Suzanne_2   
Vickie        assign_Vickie_0    assign_Vickie_1    assign_Vickie_2   
Wendie        assign_Wendie_0    assign_Wendie_1    assign_Wendie_2   
Zoe              assign_Zoe_0       assign_Zoe_1       assign_Zoe_2   

                                                                     \
All_Shifts                 3                  4                  5    
All_Nurses                                                            
Anne            assign_Anne_3      assign_Anne_4      assign_Anne_5   
Bethanie    assign_Bethanie_3  assign_Bethanie_4  assign_Bethanie_5   
Betsy          assign_Betsy_3     assign_Betsy_4     assign_Betsy_5   
Cathy          assign_Cathy_3     assign_Cathy_4     assign_Cathy_5   
Cecilia      assign_Cecilia_3   assign_Cecilia_4   assign_Cecilia_5   
Chris          assign_Chris_3     assign_Chris_4     assign_Chris_5   
Cindy          assign_Cindy_3     assign_Cindy_4     assign_Cindy_5   
David          assign_David_3     assign_David_4     assign_David_5   
Debbie        assign_Debbie_3    assign_Debbie_4    assign_Debbie_5   
Dee              assign_Dee_3       assign_Dee_4       assign_Dee_5   
Gloria        assign_Gloria_3    assign_Gloria_4    assign_Gloria_5   
Isabelle    assign_Isabelle_3  assign_Isabelle_4  assign_Isabelle_5   
Jane            assign_Jane_3      assign_Jane_4      assign_Jane_5   
Janelle      assign_Janelle_3   assign_Janelle_4   assign_Janelle_5   
Janice        assign_Janice_3    assign_Janice_4    assign_Janice_5   
Jemma          assign_Jemma_3     assign_Jemma_4     assign_Jemma_5   
Joan            assign_Joan_3      assign_Joan_4      assign_Joan_5   
Joyce          assign_Joyce_3     assign_Joyce_4     assign_Joyce_5   
Jude            assign_Jude_3      assign_Ju

In [24]:
df_sorted_shifts = df_shifts.sort_values(["Start_Time_Converted", "Duration"]).reset_index()[["Shift_ID", "Start_Time_Converted", "End_Time_Converted", "Duration"]]
df_sorted_shifts


,Shift_ID,Start_Time_Converted,End_Time_Converted,Duration
0,0,2,8,6
1,1,8,12,4
2,4,8,12,4
3,6,8,12,4
4,2,12,18,6
5,5,12,18,6
6,7,12,18,6
7,3,18,26,8
8,8,32,36,4
9,11,32,36,4


In [ ]:
number_of_overlapping_shift_constraints = 0
for shift in df_sorted_shifts.itertuples():
    for shift_2 in df_sorted_shifts.iloc[shift[0] + 1 : ].itertuples():
        if shift_2.Start_Time_Converted < shift.End_Time_Converted:
            
            for nurse_assignments in df_assigned_pivot.iloc[:, [shift.Shift_ID, shift_2.Shift_ID]].itertuples():
                mdl.add_constraint(nurse_assignments[1] + nurse_assignments[2] <=1)
                number_of_overlapping_shift_constraints += 1
        else:
            break
                
    
print(f"Number of overlapping shift constraints: {number_of_overlapping_shift_constraints}")    
    


Number of overlapping shift constraints: 640


In [ ]:
def make_var(row:tuple, varname_ftm):
    return mdl.continuous_var(name=varname_ftm % row.name, lb=0, ub=40)

df_nurses['Worktime'] = df_nurses.apply(lambda r: make_var(r, "worktime_%s"), axis=1)

In [47]:
df_nurses

,Seniority,Qualification,Pay_Rate,Worktime
Name,,,,
Anne,11,1,25,worktime_Anne
Bethanie,4,5,28,worktime_Bethanie
Betsy,2,2,17,worktime_Betsy
Cathy,2,2,17,worktime_Cathy
Cecilia,9,5,38,worktime_Cecilia
Chris,11,4,38,worktime_Chris
Cindy,5,2,21,worktime_Cindy
David,1,2,15,worktime_David
Debbie,7,2,24,worktime_Debbie


In [ ]:
for nurse, nurse_assignments in df_assigned.groupby(level='All_Nurses'):
    mdl.add_constraint(df_nurses.Worktime[nurse] == mdl.dot(nurse_assignments.Assigned, df_shifts.Duration))
    


worktime_Anne == 6assign_Anne_0+4assign_Anne_1+6assign_Anne_2+8assign_Anne_3+4assign_Anne_4+6assign_Anne_5+4assign_Anne_6+6assign_Anne_7+4assign_Anne_8+6assign_Anne_9+8assign_Anne_10+4assign_Anne_11+6assign_Anne_12+4assign_Anne_13+6assign_Anne_14+8assign_Anne_15+6assign_Anne_16+4assign_Anne_17+6assign_Anne_18+8assign_Anne_19+4assign_Anne_20+6assign_Anne_21+6assign_Anne_22+4assign_Anne_23+6assign_Anne_24+8assign_Anne_25+4assign_Anne_26+6assign_Anne_27+6assign_Anne_28+4assign_Anne_29+6assign_Anne_30+8assign_Anne_31+4assign_Anne_32+6assign_Anne_33+10assign_Anne_34+8assign_Anne_35+6assign_Anne_36+10assign_Anne_37+8assign_Anne_38+6assign_Anne_39+2assign_Anne_40
worktime_Bethanie == 6assign_Bethanie_0+4assign_Bethanie_1+6assign_Bethanie_2+8assign_Bethanie_3+4assign_Bethanie_4+6assign_Bethanie_5+4assign_Bethanie_6+6assign_Bethanie_7+4assign_Bethanie_8+6assign_Bethanie_9+8assign_Bethanie_10+4assign_Bethanie_11+6assign_Bethanie_12+4assign_Bethanie_13+6assign_Bethanie_14+8assign_Bethanie_15+6ass

In [40]:
df_assigned

Assigned
All_Nurses All_Shifts               
Anne       0           assign_Anne_0
           1           assign_Anne_1
           2           assign_Anne_2
           3           assign_Anne_3
           4           assign_Anne_4
...                              ...
Zoe        36          assign_Zoe_36
           37          assign_Zoe_37
           38          assign_Zoe_38
           39          assign_Zoe_39
           40          assign_Zoe_40

[1312 rows x 1 columns]

In [49]:
solution = mdl.solve(log_output = True, time_limit=300)
assert solution, "Solve failed"

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Found incumbent of value 0.000000 after 0.00 sec. (0.05 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.06 ticks)
Parallel b&c, 20 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.06 ticks)
